Imporing Libraries

In [ ]:
import jax.numpy as np
from jax.scipy.linalg import cholesky

a = np.array([
    [1, 0.8],
    [0.8, 1],
])

U = cholesky(a) 

In [ ]:
from jax import random as rnd

key = rnd.PRNGKey(42)
init_cov = rnd.normal(key, shape=(5, 5))  # let's make a 5x5 covariance matrix

def transform_to_covariance_matrix(sq_mat):
    U = np.triu(sq_mat)
    U_T = np.transpose(U)
    return np.dot(U_T, U)

Importing multivariate_normal

In [ ]:
from jax.scipy.stats import multivariate_normal
from jax import vmap

def loglike(params, data):
    mu, untransformed_cov = params
    cov = transform_to_covariance_matrix(untransformed_cov)
    def logpdf_func(datum):
        """logpdf of multivariate normal for one datum."""
        return multivariate_normal.logpdf(mu, cov, datum)
    logp = vmap(logpdf_func)(data)
    return np.sum(logp)

In [ ]:
from jax import grad
dloglike = grad(loglike)

In [ ]:
import jax.random as jran
mu = jran.normal(key, shape=(5,))
untransformed_cov = jran.normal(key, shape=(5, 5))
params = mu, untransformed_cov  # package them up into a convenient variable.

In [ ]:
from jax import jit
from jax.experimental.optimizers import adam

init, update, get_params = adam(step_size=0.005)
get_params = jit(get_params); update = jit(update)
dloglike = jit(dloglike); loglike = jit(loglike)

state = init(params)
for i in range(300):
    params = get_params(state)
    # g = dloglike(params, data)
    # state = update(i, g, state)
mu_opt, untransformed_cov_opt = get_params(state)
cov_opt = transform_to_covariance_matrix(untransformed_cov_opt)

Resources


*   https://jax.readthedocs.io/en/latest/_autosummary/jax.random.multivariate_normal.html
*   https://en.wikipedia.org/wiki/Multivariate_normal_distribution

